# Dipole Radiation

Any accelerating charge produces electromagnetic radiation.  If the charges in a neutral object — like an atom or a radio antenna — are undergoing periodic motion, we can approximate the system as an **oscillating electric dipole**.  As discussed in Chapter 9 of _CER_ or [Chapter 28 of the Feynman Lectures](https://www.feynmanlectures.caltech.edu/I_28.html), the __radiation field__ at point $\vec{r}$ in the $xz$-plane far from a dipole oscillating along the $z$-axis at the origin is

$$\vec{E}(\vec{r}, t) = \dfrac{1}{c^2}
\, \omega^2 p_0 \sin \theta
\, \dfrac{\cos \omega(t - r/c)}{r}
\, (\hat{e}_x \cos \theta - \hat{e}_z \sin \theta)$$

The goal of this exercise is to tame this expression and create a movie of an oscillating electric field.

First, the variables:

| Symbol | Meaning |
| --- | --- |
| $\omega$ | angular frequency of oscillation |
| $p_0$ | maximum dipole moment |
| $c$ | speed of light |
| $r$ | distance from origin to point of interest |
| $t$ | current time |
| $\theta$ | angle from $y$-axis to point of interest |

The final term in parentheses is a unit vector that gives the direction of the field.

### Warm Up

Our goal is to watch the electromagnetic waves propagate outward from the dipole as it oscillates.  This means we need to choose the time increments and spatial grid of points appropriately.  We need to ...

- Determine the duration of the simulation if we want to watch 10 complete cycles of oscillation.

- Determine the size of the grid if we want to see at least 5 wavelengths in each direction.

- Determine a reasonable time increment `dt` and grid spacing `dx` for simulating red light with a wavelength of $700 \ {\tt nm}$.

We do not have the computer hardware of lifespan to simulate a 1 billion by 1 billion grid of points for 10 billion time steps!

___Solution:___

- The wavelength of the light is $\lambda = 7.0 \, 10^{-7} \ {\tt m}$.

- The period of a wave is $T = \dfrac{1}{f} = \dfrac{\lambda}{c}$.  For red light, this works out to $T \approx 2.335 \cdot 10^{-15} \ {\tt s}$.

___Dimensional Analysis:___

These values are very small, and the frequency and speed of light are very large. We might run into roundoff error if we are not careful.  Instead, let's choose our units so that numerical values are on the order of 1.

First, note that we can rewrite the argument of the cosine function as follows:
$$\omega(t - R/c) = 2\pi \left(\dfrac{t}{T} - \dfrac{r}{\lambda}\right)$$
If we measure time in femtoseconds and distance in micrometers, all numbers in this expression will be of order 1.

Second, we can now rewrite the expression for the electric field as follows:
$$\vec{E}(\vec{r}, t)
= E_0 \, \dfrac{\lambda}{r} \, \cos 2\pi \left( \dfrac{t}{T} - \dfrac{r}{\lambda}\right)
\, (\hat{e}_x \cos \theta - \hat{e}_z \sin \theta)  \sin \theta
$$
where
$$E_0 = \dfrac{\omega^2 p_0}{\lambda c^2} = 4 \pi^2 \dfrac{p_0}{\lambda^3}$$

Different values of $E_0$ change the overall magnitude of the electric field, but not the relative size of the components at different points.  Thus, we can set $E_0 = 1$ during the calculation, and we can restore the units if we need to compute forces at another time.

This type of dimensional analysis can greatly simplify codiing and often leads to more accurate results because you do not "forget" a factor $10^9$ somewhere in the code.

Now, let's make a movie!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import system
from IPython.display import HTML, clear_output

## Linear Dipole

Our first simulation will illustrate the radiation field of a dipole oscillating along the z-axis.  This is much like the field created by a single radio tower.  Before you run the following cells, make some predictions.

- What do you expect the field to look like?
- In what directions will the strength of the field be strongest? Weakest?

***Write your predictions below.***

I'm not really sure what to expect without seeing a plot first, but I'm imagining that, as the dipole direction switches, so will the direction of the field. The field should increase in strength as it approaches the location of the dipole. 

In [7]:
from scipy.constants import c as speed_of_light

# Wavelength in m.
lambda0 = 700e-9

# Wavelength in um.
wavelength = lambda0 * 1e6

# Period in fs.
period = 1e15 * lambda0 / speed_of_light

# Define grid of points and array of times for plotting.
t_max = period
dt = period / 64
t_values = np.arange(0, t_max + dt, dt)
N = len(t_values)

x_max = 3 * wavelength
dx = wavelength / 10
coordinates = np.arange(-x_max, x_max + dx, dx)

X,Z = np.meshgrid(coordinates, coordinates)
R = np.sqrt(X**2 + Z**2 + 1e-6)
theta = np.arctan2(X,Z)

# Define field strength and cutoff for rescaling large values.
E0 = 2.0
E_cut = 1.0
R_cut = 0.5*wavelength

# Remove any existing movie frames.
system('rm -f *-linear-dipole.jpg')

# It is essential that the frames be named in alphabetical order.
# {:03d} will display integers with three digits and insert zeros if needed:
# '000_movie.jpg', '001_movie.jpg', etc.
file_name = "{:03d}-linear-dipole.jpg"


plt.figure(figsize=(6,6), dpi=200)
# Generate frames and save each figure as a separate .jpg file.
for i, t in enumerate(t_values):
    # Compute magnitude of field.
    ## Dipole oscillating along z-axis.
    #pz = np.cos(2*np.pi * (t/period))
    #px = 0
    #E =  -E0 * wavelength * np.sin(theta) * np.cos(2*np.pi * (t/period - R/wavelength)) / R
    
    ## Dipole oscillating along x-axis.
    pz = 0
    px = np.cos(2*np.pi * (t/period))    
    E =  E0 * wavelength * np.cos(theta) * np.cos(2*np.pi * (t/period - R/wavelength)) / R

    # Use this for the oscillating dipole.
    Ex = E * np.cos(theta)
    Ez = -E * np.sin(theta)

    # Rescale large values.
    Ex[R<R_cut] = 0
    Ez[R<R_cut] = 0
    E = np.sqrt(Ex**2 + Ez**2) 
    bad = E > E_cut
    Ex[bad] = E_cut * Ex[bad] / E[bad]
    Ez[bad] = E_cut * Ez[bad] / E[bad]
    
    # Make plot and save to file.
    plt.quiver(X, Z, Ex, Ez, pivot='middle', lw=1, scale=40)
    plt.quiver(0,0,px,pz, pivot='middle', color='red', lw=1, scale=10)
    plt.xlabel('X [um]')
    plt.ylabel('Z [um]')
    plt.savefig(file_name.format(i))  # save current plot

    # Clear plot for next frame.
    plt.cla()
    
    print("Completed frame %d of %d.\r" % (i+1,N), end='')

plt.close()

In [8]:
!ffmpeg -y -i %03d-linear-dipole.jpg -pix_fmt yuv420p linear-movie.mp4

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoo

frame=   65 fps=0.0 q=-1.0 Lsize=    2309kB time=00:00:02.48 bitrate=7627.3kbits/s speed= 2.8x    
video:2308kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.061107%
[libx264 @ 0x134605ab0] frame I:1     Avg QP:22.76  size:135717
[libx264 @ 0x134605ab0] frame P:43    Avg QP:24.45  size: 44739
[libx264 @ 0x134605ab0] frame B:21    Avg QP:28.72  size: 14423
[libx264 @ 0x134605ab0] consecutive B-frames: 56.9%  0.0%  0.0% 43.1%
[libx264 @ 0x134605ab0] mb I  I16..4: 27.6% 19.7% 52.6%
[libx264 @ 0x134605ab0] mb P  I16..4:  0.1%  0.1%  0.2%  P16..4: 12.2% 12.9% 17.6%  0.0%  0.0%    skip:56.9%
[libx264 @ 0x134605ab0] mb B  I16..4:  0.1%  0.1%  0.0%  B16..8:  6.2%  4.8%  7.0%  direct: 9.5%  skip:72.5%  L0:45.1% L1:36.1% BI:18.8%
[libx264 @ 0x134605ab0] 8x8 transform intra:20.6% inter:5.1%
[libx264 @ 0x134605ab0] coded y,uvDC,uvAC intra: 37.9% 0.2% 0.2% inter: 18.1% 0.0% 0.0%
[libx264 @ 0x134605ab0] i16 v,h,dc,p: 76% 15%  8%  0%
[libx264 @ 0x134605ab0] i8 v,h,dc,

In [9]:
# Play the movie within the notebook.
HTML("""
    <video width="800" height="800" alt="test" controls loop>
        <source src=%s type="video/mp4">
    </video>
""" % "linear-movie.mp4")

### Questions

Describe the field pattern.

- What does the field to look like?
- In what directions will the strength of the field be strongest? Weakest?

***Report your observations below.***

The field emulates from the dipole radially in all directions, but its strength wanes (almost completely to zero) along the axis of dipole oscillation. This makes sense, as the field will be generated perpendicular to the direction of propogation. 

Would you describe this radiation as a "plane wave"?  Why or why not?

***Share your opinion below.***

I think so, but I'm not confident in my answer. If we imagine the plot in 3D, a spherical wave front would be formed with uniform plane waves over defined areas. 

**Different Axis**

How would you expect the pattern to change if you rotated the dipole by 90°?

***Report your prediction below.***

Rotating the dipole should rotate the entire plot by 90 degrees, without changing the behaviour of the wave. That is, the field strength should still approach zero in a direction parallel to the dipole. 

The code above contains instructions for viewing a dipole oscillating along the x-axis.

Comment out the three lines below
```
    ## Dipole oscillating along z-axis.
```
Uncomment the three lines below
```
    ## Dipole oscillating along x-axis.
```
Rerun the simulation.

- Describe what you see.
- Assess your predictions.

***Report your observations below.***

This matched my prediction - the dipole was tipped on its side as were the waves eminating from the dipole. 

## Rotating Dipole

Instead of a single dipole oscillating along the $y$-axis, analyze a dipole that is rotating in the $xy$-plane.  The dipole moment is

$$\vec{p}(t) = p_0 \left(\hat{e}_x \cos \omega t + \hat{e}_z \sin \omega t \right)$$

This is equivalent to two _linear_ dipoles at right angles to each other, oscillating $90^\circ$ out of phase.

The code below simply adds together the fields of the two dipoles you just observed.  Before you run it, make some predictions.

- What do you expect the field to look like?
- In what directions will the strength of the field be strongest? Weakest?

***Write your predictions below.***

I predict that the field strength will still be lowest at angles parallel to the two dipoles, and it will be strongest at angles in-between the two dipoles (around 45 degrees + incrememnts 90 degrees around the circle).

In [49]:
from scipy.constants import c as speed_of_light

# Wavelength in m.
lambda0 = 700e-9

# Wavelength in um.
wavelength = lambda0 * 1e6

# Period in fs.
period = 1e15 * lambda0 / speed_of_light

# Define grid of points and array of times for plotting.
t_max = period
dt = period / 64
t_values = np.arange(0, t_max + dt, dt)
N = len(t_values)

x_max = 3 * wavelength
dx = wavelength / 10
coordinates = np.arange(-x_max, x_max + dx, dx)

X,Z = np.meshgrid(coordinates, coordinates)
R = np.sqrt(X**2 + Z**2 + 1e-6)
theta = np.arctan2(X,Z)

# Define field strength and cutoff for rescaling large values.
E0 = 2.0
E_cut = 1.0
R_cut = 0.5*wavelength

# Remove any existing movie frames.
system('rm -f *-rotating-dipole.jpg')

# It is essential that the frames be named in alphabetical order.
# {:03d} will display integers with three digits and insert zeros if needed:
# '000_movie.jpg', '001_movie.jpg', etc.
file_name = "{:03d}-rotating-dipole.jpg"


plt.figure(figsize=(6,6), dpi=200)
# Generate frames and save each figure as a separate .jpg file.
for i, t in enumerate(t_values):
    # Plot the dipole that is creating the fields.
    pz = -np.cos(2*np.pi * (t/period))
    px = 0.5*np.sin(2*np.pi * (t/period))

    # Compute magnitudes of fields.
    E1 = E0 * wavelength * np.sin(theta) * np.cos(2*np.pi * (t/period - R/wavelength)) / R
    E2 = -0.5*E0 * wavelength * np.cos(theta) * np.sin(2*np.pi * (t/period - R/wavelength)) / R

    Ex = (E1 + E2) * np.cos(theta)
    Ez = -(E1 + E2) * np.sin(theta)

    # Rescale large values.
    Ex[R<R_cut] = 0
    Ez[R<R_cut] = 0
    E = np.sqrt(Ex**2 + Ez**2) 
    bad = E > E_cut
    Ex[bad] = E_cut * Ex[bad] / E[bad]
    Ez[bad] = E_cut * Ez[bad] / E[bad]

    # Make plot and save to file.
    plt.quiver(X, Z, Ex, Ez, pivot='middle', lw=1, scale=40)
    plt.quiver(0,0,px,pz, pivot='middle', color='red', lw=1, scale=10)
    plt.xlabel('X [um]')
    plt.ylabel('Z [um]')
    plt.savefig(file_name.format(i))  # save current plot

    # Clear plot for next frame.
    plt.cla()
    
    print("Completed frame %d of %d.\r" % (i+1,N), end='')

plt.close()

In [50]:
!ffmpeg -y -i %03d-rotating-dipole.jpg -pix_fmt yuv420p rotating-movie.mp4

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoo

frame=   65 fps=0.0 q=-1.0 Lsize=    2467kB time=00:00:02.48 bitrate=8149.4kbits/s speed= 2.6x    
video:2466kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.060201%
[libx264 @ 0x14f6077d0] frame I:1     Avg QP:23.01  size:153926
[libx264 @ 0x14f6077d0] frame P:37    Avg QP:25.37  size: 51163
[libx264 @ 0x14f6077d0] frame B:27    Avg QP:29.10  size: 17676
[libx264 @ 0x14f6077d0] consecutive B-frames: 44.6%  0.0%  0.0% 55.4%
[libx264 @ 0x14f6077d0] mb I  I16..4: 26.3% 20.6% 53.1%
[libx264 @ 0x14f6077d0] mb P  I16..4:  0.1%  0.1%  0.2%  P16..4: 11.6% 14.9% 20.8%  0.0%  0.0%    skip:52.3%
[libx264 @ 0x14f6077d0] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  6.0%  5.8%  8.6%  direct:11.0%  skip:68.4%  L0:45.8% L1:33.9% BI:20.3%
[libx264 @ 0x14f6077d0] 8x8 transform intra:21.1% inter:5.2%
[libx264 @ 0x14f6077d0] coded y,uvDC,uvAC intra: 41.0% 0.7% 0.7% inter: 19.4% 0.1% 0.1%
[libx264 @ 0x14f6077d0] i16 v,h,dc,p: 80% 14%  7%  0%
[libx264 @ 0x14f6077d0] i8 v,h,dc,

In [51]:
# Play the movie within the notebook.
HTML("""
    <video width="800" height="800" alt="test" controls loop>
        <source src=%s type="video/mp4">
    </video>
""" % "rotating-movie.mp4")

### Questions

Describe the field pattern.

- What does the field to look like?
- In what directions will the strength of the field be strongest? Weakest?

***Report your observations below.***

The field waves eminate around the dipole in all directions. There is no apparent difference in strength at certain angles. Instead, a swirling pattern is generated, and the strength of the field wanes as distance from the dipole increases. 

Would you describe this radiation as a "plane wave"?  Why or why not?

***Share your opinion below.***

In this case, no, since the waves do not form neat, circular shapes but spiral around the dipole. Thus, if we imagine a 3D spherical shape of constant radius, the strength of the field may be different when taken at different sections of surface area.

Modify the code above to make the dipole rotate in the opposite direction ***and*** produce the correct radiation pattern.

**Hint:** It only requires two minus signs ...

- Describe your attempts and your results.
- Why are only two minus signs required?

***Report your observations below.***

By reversing the sign of the px variable for the dipole, I was able to reverse the direction of rotation. However, this did not result in a "correct" radiation pattern. I will watch the recording from Tuesday's class to learn more about how other students were able to get the correct pattern.

What would happen if the dipole along the x-axis were **half as large** as the dipole along the z-axis?

- How do you think this would change the field pattern?  Why?

***Report your predictions below.***

I'm not sure if this would drastically change the field pattern but it should change the field strength since we're reducing the magnitude of one of its components by half. I still expect to see the same swirling pattern as before. 

Modify the code above so that the dipole along the  x-axis is **half as large** as that along the z-axis.  You will need to modify `px` and `E2`.

- How does this change the field pattern?

***Report your observations below.***

I was correct, but the fields also seem to wane more quickly under these conditions. This makes sense, as the initial magnitude should be lower. 

# Reflection and Summary

- What are the major takeaways of this assignment for you?
- What was the most difficult part of this assignment?
- What was the most interesting part of this assignment?
- What questions do you have?

***Include your response below.***

I really enjoyed the explanation at the beginning of the assignment that clarified how we can break down a seemingly complex system into much simpler components. For example, modeling an oscillating dipole instead of an accelerating charge and breaking down the timescale to numerical values are on the order of one. As a novice coder, this type of analysis seems incredibly useful to speed up processing and computation. 

The most difficult part was obtaining the correct field pattern for the rotating dipole system. I will review the lecture recording from Tuesday to see how that was accomplished. Otherwise, I found the shape of those plots to be the most intriguing. The spiral shape makes much more sense after plotting the system and thinking more deeply about its inputs. 

My main question deals with getting the correct pattern response for the rotating dipole system. 

# Challenge Problems

The challenge problems below ask you to extend the simulation above.  

## Near Fields

The simulations above only include the radiation field — the portion of the field that decreases as $1/r$.  Far from the source, this is the only part that matters.  Near the source, the other contributions to the electric field are the most important.

Refer to Equation (9.39) in _CER_.  Try to plot the full electric field for the dipole

$$\vec{p} = p_0 \, \hat{e}_z \, \cos \omega t$$


## Multiple Sources

In the code above, all of the dipoles are at the origin.  Adapt the code above to simulate the field of two dipoles at different locations.


## 3D Plots

Adapt the code to plot the field of a linear dipole or a rotating dipole in three dimensions.


## Plot Everything

Add the magnetic field and the Poynting vector to a 3D simulation!

In [ ]:
### Replace with your code.

## A note on coordinates and formulae

The simulation above uses spherical coordinates to match equations in textbooks.  However, this is not always convenient for simulating problems on a grid.  For instance, the equations for the fields assume the dipoles are at the origin.  If there are two dipoles at different locations, "$\hat{e}_r$" and "$\hat{e}_\theta$" in the formulas are ***not*** the same vector for both dipoles.

It might be easier to work in Cartesian coordinates.  The following relations may be useful if you attempt the challenge problems. If the location of the dipole is $(x_0, y_0, z_0)$, then the textbook formulae translate as follows.

\begin{align*}
r &= \sqrt{(x-x_0)^2 + (y-y_0)^2 + (z-z_0)^2} \\
\cos \theta & = \dfrac{z-z_0}{r} \\
\sin \theta &= \dfrac{\sqrt{(x-x_0)^2 + (y-y_0)^2}}{r} \\
\cos \phi &= \dfrac{x-x_0}{\sqrt{(x-x_0)^2+(y-y_0)^2}} \\
\sin \phi &= \dfrac{y-y_0}{\sqrt{(x-x_0)^2+(y-y_0)^2}} \\
\hat{e}_r &= \hat{e}_x \, \sin \theta \cos \phi + \hat{e}_y \, \sin \theta \sin \phi + \hat{e}_z \, \cos \theta \\
\hat{e}_\theta &= \hat{e}_x \, \cos \theta \cos \phi + \hat{e}_y \, \cos \theta \sin \phi - \hat{e}_z \sin \theta \\
\hat{e}_\phi &= -\hat{e}_x \, \sin \phi + \hat{e}_y \, \cos \phi
\end{align*}